In [1]:
"""
Use of same experimental setup as for our Probabilistic Suffix Prediction.

Reimplementation for comparison: 
- Paper: Camargo, Manuel, Marlon Dumas, and Oscar González-Rojas. "Learning accurate LSTM models of business processes." International Conference on Business Process Management. Cham: Springer International Publishing, 2019.
- Github (code) from: https://github.com/AdaptiveBProcess/GenerativeLSTM/tree/master/
"""

'\nUse of same experimental setup as for our Probabilistic Suffix Prediction.\n\nReimplementation for comparison: \n- Paper: Camargo, Manuel, Marlon Dumas, and Oscar González-Rojas. "Learning accurate LSTM models of business processes." International Conference on Business Process Management. Cham: Springer International Publishing, 2019.\n- Github (code) from: https://github.com/AdaptiveBProcess/GenerativeLSTM/tree/master/\n'

# Imports

In [2]:
import importlib
import sys
import torch

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')

# Data

### Load Data Files

In [3]:
# Path to your pickle file (saved with torch.save)
file_path_train = '../../../../../../encoded_data/compare_camargo/BPIC_2017_all_5_train.pkl'
# Load the dataset using torch.load
bpic17_train_dataset = torch.load(file_path_train, weights_only=False)
# Check the type of the loaded dataset
print(type(bpic17_train_dataset))

# Path to your pickle file (saved with torch.save)
file_path_val = '../../../../../../encoded_data/compare_camargo/BPIC_2017_all_5_val.pkl'
# Load the dataset using torch.load
bpic17_val_dataset = torch.load(file_path_val, weights_only=False)
# Check the type of the loaded dataset
print(type(bpic17_val_dataset))


<class 'event_log_loader.new_event_log_loader.EventLogDataset'>


<class 'event_log_loader.new_event_log_loader.EventLogDataset'>


### Train Data Insights

In [4]:
# BPIC 2017 Dataset Categories, Features:
bpic17_all_categories = bpic17_train_dataset.all_categories

bpic17_all_categories_cat = bpic17_all_categories[0]
print(bpic17_all_categories_cat)

bpic17_all_categories_num = bpic17_all_categories[1]
print(bpic17_all_categories_num)

for i, cat in enumerate(bpic17_all_categories_cat):
     print(f"sepsis(5) Categorical feature: {cat[0]}, Index position in categorical data list: {i}")
     print(f"sepsis (5) Total Amount of Category labels: {cat[1]}")

print('\n')    

for i, num in enumerate(bpic17_all_categories_num):
     print(f"BPIC17 (1) Numerical feature: {num[0]}, Index position in categorical data list: {i}")
     print(f"BPIC17 (1) Amount Numerical: {num[1]}")
     
# Get concept_name id:
concept_name = 'concept:name'
concept_name_id = [i for i, cat in enumerate(bpic17_all_categories[0]) if cat[0] == concept_name][0]
print("ID concet name in cat list: ", concept_name_id)

# Output size
concept_name = 'concept:name'
concept_name_size = [cat[1] for _, cat in enumerate(bpic17_all_categories[0]) if cat[0] == concept_name][0]
print("ID concet name in cat list: ", concept_name_size)

# Id of EOS token in activity
eos_value = 'EOS'
eos_id = [v for k, v in bpic17_all_categories[0][concept_name_id][2].items() if k == eos_value][0]
# Get EOS id of concept name list:
print("ID EOS in concept name tensor: ", eos_id)


[('concept:name', 28, {'A_Accepted': 1, 'A_Cancelled': 2, 'A_Complete': 3, 'A_Concept': 4, 'A_Create Application': 5, 'A_Denied': 6, 'A_Incomplete': 7, 'A_Pending': 8, 'A_Submitted': 9, 'A_Validating': 10, 'EOS': 11, 'O_Accepted': 12, 'O_Cancelled': 13, 'O_Create Offer': 14, 'O_Created': 15, 'O_Refused': 16, 'O_Returned': 17, 'O_Sent (mail and online)': 18, 'O_Sent (online only)': 19, 'W_Assess potential fraud': 20, 'W_Call after offers': 21, 'W_Call incomplete files': 22, 'W_Complete application': 23, 'W_Handle leads': 24, 'W_Personal Loan collection': 25, 'W_Shortened completion ': 26, 'W_Validate application': 27}), ('org:resource', 151, {'EOS': 1, 'User_1': 2, 'User_10': 3, 'User_100': 4, 'User_101': 5, 'User_102': 6, 'User_103': 7, 'User_104': 8, 'User_105': 9, 'User_106': 10, 'User_107': 11, 'User_108': 12, 'User_109': 13, 'User_11': 14, 'User_110': 15, 'User_111': 16, 'User_112': 17, 'User_113': 18, 'User_114': 19, 'User_115': 20, 'User_116': 21, 'User_117': 22, 'User_118': 23, 

### Input Features for Encoder and Decoder

In [5]:
# Create lists with name of Model features (input)
model_feat_cat = []
model_feat_num = []
for cat in bpic17_all_categories_cat:
    model_feat_cat.append(cat[0])
for num in bpic17_all_categories_num:
    model_feat_num.append(num[0])
model_feat = [model_feat_cat, model_feat_num]
print("Input features encoder: ", model_feat)

Input features encoder:  [['concept:name', 'org:resource'], ['case_elapsed_time']]


# Model

In [6]:
import joinLSTM.model
importlib.reload(joinLSTM.model)
from joinLSTM.model import FullShared_Join_LSTM

"""
Specific model parameters from paper: 
"""

# Size hidden layer
hidden_size= 50

# Number of LSTM cells
num_layers = 1

# STANDARD: One numerical output to predict
input_size = 1

# Hans Weytjens LSTM model
model = FullShared_Join_LSTM(data_set_categories=bpic17_all_categories,
                             hidden_size=hidden_size,
                             num_layers=num_layers,
                             model_feat=model_feat,
                             input_size=input_size,
                             output_size_act=concept_name_size)

Data set categories:  ([('concept:name', 28, {'A_Accepted': 1, 'A_Cancelled': 2, 'A_Complete': 3, 'A_Concept': 4, 'A_Create Application': 5, 'A_Denied': 6, 'A_Incomplete': 7, 'A_Pending': 8, 'A_Submitted': 9, 'A_Validating': 10, 'EOS': 11, 'O_Accepted': 12, 'O_Cancelled': 13, 'O_Create Offer': 14, 'O_Created': 15, 'O_Refused': 16, 'O_Returned': 17, 'O_Sent (mail and online)': 18, 'O_Sent (online only)': 19, 'W_Assess potential fraud': 20, 'W_Call after offers': 21, 'W_Call incomplete files': 22, 'W_Complete application': 23, 'W_Handle leads': 24, 'W_Personal Loan collection': 25, 'W_Shortened completion ': 26, 'W_Validate application': 27}), ('org:resource', 151, {'EOS': 1, 'User_1': 2, 'User_10': 3, 'User_100': 4, 'User_101': 5, 'User_102': 6, 'User_103': 7, 'User_104': 8, 'User_105': 9, 'User_106': 10, 'User_107': 11, 'User_108': 12, 'User_109': 13, 'User_11': 14, 'User_110': 15, 'User_111': 16, 'User_112': 17, 'User_113': 18, 'User_114': 19, 'User_115': 20, 'User_116': 21, 'User_117

/home/LordKunkler/PrimePPM/.venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


# Training Configuration

In [7]:
import training.train
importlib.reload(training.train)
from training.train import Training

from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(comment="Full_bpic17_camargo_act")

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
Parameter of Probabilistic Suffix Prediction experimental design, to ensure fair comparison:
"""

# Start learning rate
learning_rate = 1e-6

# Optimizer and Scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, min_lr=1e-10)

# Epochs
num_epochs = 100

# Batch of model input
batch_size = 256

# shuffle data
shuffle = True

optimize_values = {"optimizer":optimizer,
                   "scheduler": scheduler,
                   "epochs":num_epochs,
                   "mini_batches":batch_size,
                   "shuffle": shuffle}

number_tasks = len(model_feat)

trainer = Training(model=model,
                   device=device,
                   data_train=bpic17_train_dataset,
                   data_val=bpic17_val_dataset,
                   optimize_values=optimize_values,
                   concept_name_id=concept_name_id,
                   eos_id=eos_id,
                   writer=writer,
                   save_model_n_th_epoch=1,
                   saving_path="BPIC17_camargo_act_1_suffix_length5.pkl")

# Train the model:
trainer.train()

Device:  cuda
Optimizer:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-06
    maximize: False
    weight_decay: 0
)
Scheduler:  <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x7fef93b24440>
Epochs:  100
Mini baches:  256
Shuffle batched dataset:  True


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/100], Learning Rate: 1e-06
Training: Avg Attenuated Training Loss: 3.3302


Validation: Avg Validation Loss: 3.3259
Validation Loss for Scheduler: 3.3259
saving model


Epoch [2/100], Learning Rate: 1e-06
Training: Avg Attenuated Training Loss: 3.3197


...

Validation: Avg Validation Loss: 2.8257
Validation Loss for Scheduler: 2.8257
saving model


Epoch [99/100], Learning Rate: 1e-06
Training: Avg Attenuated Training Loss: 2.8244


Validation: Avg Validation Loss: 2.8255
Validation Loss for Scheduler: 2.8255
saving model


Epoch [100/100], Learning Rate: 1e-06
Training: Avg Attenuated Training Loss: 2.8243


Validation: Avg Validation Loss: 2.8253
Validation Loss for Scheduler: 2.8253
saving model
Training complete.
Model saved to path: BPIC17_camargo_act_1_suffix_length5.pkl
